In [ ]:
import pandas as pd
import fasttext
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from collections import Counter
import pycountry


### Traitement du fichier final sommeil.csv

In [2]:
final = pd.read_csv("sommeil.csv")
final

,Titre,Date de publication,Lien,Résumé,Mots-clés,Auteurs,Affiliations,PMID,Maladie
0,Sleep debt and obesity.,2014 Aug;46(5):264-72.,https://pubmed.ncbi.nlm.nih.gov//25012962/,Short sleep duration has been shown to be asso...,Energy expenditure; energy intake; epidemiolog...,"Virginie Bayon, 1, Damien Leger, Danielle Gome...","1 Université Paris Descartes, Sorbonne Paris C...",25012962,Obesite
1,Insufficient sleep in adolescents and young ad...,2014 Sep;134(3):e921-32.,https://pubmed.ncbi.nlm.nih.gov//25157012/,Chronic sleep loss and associated sleepiness a...,adolescents; caffeine; car crashes; media use;...,"Judith Owens, Adolescent Sleep Working Group, ...",Affiliations non disponibles,25157012,Obesite
2,Role of sleep and sleep loss in hormonal relea...,2010:17:11-21.,https://pubmed.ncbi.nlm.nih.gov//19955752/,"Compared to a few decades ago, adults, as well...",Mots-clés non disponibles,"Rachel Leproult, Eve Van Cauter",Affiliations non disponibles,19955752,Obesite
3,"Sleep patterns, diet quality and energy balance.",2014 Jul:134:86-91.,https://pubmed.ncbi.nlm.nih.gov//24051052/,There is increasing evidence showing that slee...,Diet; Eating behavior; Energy; Food; Obesity; ...,"Jean-Philippe Chaput, 1",1 Healthy Active Living and Obesity Research G...,24051052,Obesite
4,Social jetlag and obesity.,2012 May 22;22(10):939-43.,https://pubmed.ncbi.nlm.nih.gov//22578422/,Obesity has reached crisis proportions in indu...,Mots-clés non disponibles,"Till Roenneberg, 1, Karla V Allebrandt, Martha...","1 Institute for Medical Psychology, University...",22578422,Obesite
...,...,...,...,...,...,...,...,...,...
64789,Withdrawal Notice: A New Perspective on the Tr...,2022 Feb 17.,https://pubmed.ncbi.nlm.nih.gov//35176983/,The article has been withdrawn at the request ...,Mots-clés non disponibles,"Esra Küpeli Akkol, 1, Hilal Bardakcı, 2, Çiğde...",1 Department of Pharmacognosy Faculty of Pharm...,35176983,Alzheimer
64790,Enhancement of cognitive and neural functions ...,2014 Apr 28:8:69.,https://pubmed.ncbi.nlm.nih.gov//24808834/,Public awareness of cognitive health is fairly...,brain plasticity; cognition; cognitive trainin...,"Sandra B Chapman, 1, Raksha A Mudar, 2","1 Center for BrainHealth®, The University of T...",24808834,Alzheimer
64791,Prevalence and associated factors of excessive...,2024 Jun;28(3):1459-1464.,https://pubmed.ncbi.nlm.nih.gov//38374476/,Objective:\n \n \n To investi...,Excessive daytime sleepiness; Older adults; Po...,"Juan Ren, 1, Rui Liu, 2, 3, Tong Zhao, 1, Jie ...","1 Department of Neurology, Shandong Provincial...",38374476,Alzheimer
64792,"Preface: Special issue ""Brain Proteostasis in ...",2023 Jul;166(1):3-6.,https://pubmed.ncbi.nlm.nih.gov//37414435/,This preface introduces the Journal of Neuroch...,Mots-clés non disponibles,"Mychael V Lourenco, 1",1 Institute of Medical Biochemistry Leopoldo d...,37414435,Alzheimer


#### Supprimer les info manquantes de type résumé, date etc

In [ ]:
final = final[
    (final["Résumé"] != "Résumé non disponible") &
    (final["Date de publication"] != "Date inconnue") &
    (final["Auteurs"] != "Auteurs non disponibles") &
    (final["Affiliations"] != "Affiliations non disponibles")
]


final


,Titre,Date de publication,Lien,Résumé,Mots-clés,Auteurs,Affiliations,PMID,Maladie
0,Sleep debt and obesity.,2014 Aug;46(5):264-72.,https://pubmed.ncbi.nlm.nih.gov//25012962/,Short sleep duration has been shown to be asso...,Energy expenditure; energy intake; epidemiolog...,"Virginie Bayon, 1, Damien Leger, Danielle Gome...","1 Université Paris Descartes, Sorbonne Paris C...",25012962,Obesite
3,"Sleep patterns, diet quality and energy balance.",2014 Jul:134:86-91.,https://pubmed.ncbi.nlm.nih.gov//24051052/,There is increasing evidence showing that slee...,Diet; Eating behavior; Energy; Food; Obesity; ...,"Jean-Philippe Chaput, 1",1 Healthy Active Living and Obesity Research G...,24051052,Obesite
4,Social jetlag and obesity.,2012 May 22;22(10):939-43.,https://pubmed.ncbi.nlm.nih.gov//22578422/,Obesity has reached crisis proportions in indu...,Mots-clés non disponibles,"Till Roenneberg, 1, Karla V Allebrandt, Martha...","1 Institute for Medical Psychology, University...",22578422,Obesite
7,Sleep and obesity.,2011 Jul;14(4):402-12.,https://pubmed.ncbi.nlm.nih.gov//21659802/,Purpose of review:\n \n \n Th...,Mots-clés non disponibles,"Guglielmo Beccuti, 1, Silvana Pannain","1 Department of Medicine, University of Chicag...",21659802,Obesite
8,The effect of exercise training on obstructive...,2011 Dec 1;34(12):1631-40.,https://pubmed.ncbi.nlm.nih.gov//22131599/,Study objectives:\n \n \n To ...,Actigraphy; exercise training; obstructive sle...,"Christopher E Kline, 1, E Patrick Crowley, Gar...","1 Department of Psychiatry, University of Pitt...",22131599,Obesite
...,...,...,...,...,...,...,...,...,...
64789,Withdrawal Notice: A New Perspective on the Tr...,2022 Feb 17.,https://pubmed.ncbi.nlm.nih.gov//35176983/,The article has been withdrawn at the request ...,Mots-clés non disponibles,"Esra Küpeli Akkol, 1, Hilal Bardakcı, 2, Çiğde...",1 Department of Pharmacognosy Faculty of Pharm...,35176983,Alzheimer
64790,Enhancement of cognitive and neural functions ...,2014 Apr 28:8:69.,https://pubmed.ncbi.nlm.nih.gov//24808834/,Public awareness of cognitive health is fairly...,brain plasticity; cognition; cognitive trainin...,"Sandra B Chapman, 1, Raksha A Mudar, 2","1 Center for BrainHealth®, The University of T...",24808834,Alzheimer
64791,Prevalence and associated factors of excessive...,2024 Jun;28(3):1459-1464.,https://pubmed.ncbi.nlm.nih.gov//38374476/,Objective:\n \n \n To investi...,Excessive daytime sleepiness; Older adults; Po...,"Juan Ren, 1, Rui Liu, 2, 3, Tong Zhao, 1, Jie ...","1 Department of Neurology, Shandong Provincial...",38374476,Alzheimer
64792,"Preface: Special issue ""Brain Proteostasis in ...",2023 Jul;166(1):3-6.,https://pubmed.ncbi.nlm.nih.gov//37414435/,This preface introduces the Journal of Neuroch...,Mots-clés non disponibles,"Mychael V Lourenco, 1",1 Institute of Medical Biochemistry Leopoldo d...,37414435,Alzheimer


#### Langue des articles

In [ ]:
model_path = r"C:\Users\Hendrix\Anaconda3\Lib\site-packages\fasttext\lid.176.bin"

model = fasttext.load_model(model_path)



# Fonction pour détecter la langue
def detect_language(text):
    return model.predict(text.replace("\n", " ").strip())[0][0].replace("__label__", "")

# Appliquer la détection de langue
final["Langue"] = final["Résumé"].astype(str).apply(detect_language)


c:\Users\Hendrix\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [5]:
print(final["Langue"].nunique())  # On 9 langues unique, on va juste garder les documents en anglais car plus nombreux
print(len(final[final["Langue"]=="en"]))
final=final[final["Langue"]=="en"]
final

9
60328


,Titre,Date de publication,Lien,Résumé,Mots-clés,Auteurs,Affiliations,PMID,Maladie,Langue
0,Sleep debt and obesity.,2014 Aug;46(5):264-72.,https://pubmed.ncbi.nlm.nih.gov//25012962/,Short sleep duration has been shown to be asso...,Energy expenditure; energy intake; epidemiolog...,"Virginie Bayon, 1, Damien Leger, Danielle Gome...","1 Université Paris Descartes, Sorbonne Paris C...",25012962,Obesite,en
3,"Sleep patterns, diet quality and energy balance.",2014 Jul:134:86-91.,https://pubmed.ncbi.nlm.nih.gov//24051052/,There is increasing evidence showing that slee...,Diet; Eating behavior; Energy; Food; Obesity; ...,"Jean-Philippe Chaput, 1",1 Healthy Active Living and Obesity Research G...,24051052,Obesite,en
4,Social jetlag and obesity.,2012 May 22;22(10):939-43.,https://pubmed.ncbi.nlm.nih.gov//22578422/,Obesity has reached crisis proportions in indu...,Mots-clés non disponibles,"Till Roenneberg, 1, Karla V Allebrandt, Martha...","1 Institute for Medical Psychology, University...",22578422,Obesite,en
7,Sleep and obesity.,2011 Jul;14(4):402-12.,https://pubmed.ncbi.nlm.nih.gov//21659802/,Purpose of review:\n \n \n Th...,Mots-clés non disponibles,"Guglielmo Beccuti, 1, Silvana Pannain","1 Department of Medicine, University of Chicag...",21659802,Obesite,en
8,The effect of exercise training on obstructive...,2011 Dec 1;34(12):1631-40.,https://pubmed.ncbi.nlm.nih.gov//22131599/,Study objectives:\n \n \n To ...,Actigraphy; exercise training; obstructive sle...,"Christopher E Kline, 1, E Patrick Crowley, Gar...","1 Department of Psychiatry, University of Pitt...",22131599,Obesite,en
...,...,...,...,...,...,...,...,...,...,...
64789,Withdrawal Notice: A New Perspective on the Tr...,2022 Feb 17.,https://pubmed.ncbi.nlm.nih.gov//35176983/,The article has been withdrawn at the request ...,Mots-clés non disponibles,"Esra Küpeli Akkol, 1, Hilal Bardakcı, 2, Çiğde...",1 Department of Pharmacognosy Faculty of Pharm...,35176983,Alzheimer,en
64790,Enhancement of cognitive and neural functions ...,2014 Apr 28:8:69.,https://pubmed.ncbi.nlm.nih.gov//24808834/,Public awareness of cognitive health is fairly...,brain plasticity; cognition; cognitive trainin...,"Sandra B Chapman, 1, Raksha A Mudar, 2","1 Center for BrainHealth®, The University of T...",24808834,Alzheimer,en
64791,Prevalence and associated factors of excessive...,2024 Jun;28(3):1459-1464.,https://pubmed.ncbi.nlm.nih.gov//38374476/,Objective:\n \n \n To investi...,Excessive daytime sleepiness; Older adults; Po...,"Juan Ren, 1, Rui Liu, 2, 3, Tong Zhao, 1, Jie ...","1 Department of Neurology, Shandong Provincial...",38374476,Alzheimer,en
64792,"Preface: Special issue ""Brain Proteostasis in ...",2023 Jul;166(1):3-6.,https://pubmed.ncbi.nlm.nih.gov//37414435/,This preface introduces the Journal of Neuroch...,Mots-clés non disponibles,"Mychael V Lourenco, 1",1 Institute of Medical Biochemistry Leopoldo d...,37414435,Alzheimer,en


In [6]:
final["Maladie"].unique()

array(['Obesite', 'Cancer', 'Diabete', 'Hypertension Arterielle',
       'Alzheimer'], dtype=object)

#### Supprimer les articles dont les abstract n'ont pas certains mots-clés

In [ ]:
# Définition des mots-clés
keywords_sleep = ["sleep", "sleep habits", "sleep disorder", "insomnia", "sleep deprivation", "sleep quality", "sleep apnea", "REM sleep", "non-REM sleep"]

keywords_disease = {
    "Alzheimer": ["Alzheimer's disease", "dementia", "cognitive decline", "neurodegeneration"],
    "Cancer": ["cancer", "tumor", "carcinoma", "malignant", "oncology"],
    "Diabete": ["diabetes", "hyperglycemia", "T2B", "Type 2 Diabetes"],
    "Hypertension Arterielle": ["hypertension", "high blood pressure", "cardiovascular disease"],
    "Obesite": ["obesity"] 
}

# Fonction pour filtrer
def filtre_article(resume, maladie):
    if not isinstance(resume, str):
        return False  # Exclure les valeurs NaN ou autres types

    resume_lower = resume.lower() # tout en minuscule

    # Vérification de la présence d'au moins un mot-clé lié au sommeil
    if not any(mot in resume_lower for mot in keywords_sleep):
        return False

    # Vérification de la présence d'un mot-clé lié à la maladie
    return any(mot.lower() in resume_lower for mot in keywords_disease[maladie])

# filtrer pour chaque maladie
df_filtre = pd.DataFrame()

for maladie in keywords_disease.keys():
    df_maladie = final[final["Maladie"] == maladie]
    df_maladie = df_maladie[df_maladie["Résumé"].apply(lambda x: filtre_article(x, maladie))]
    df_filtre = pd.concat([df_filtre, df_maladie], ignore_index=True)

final=df_filtre


In [8]:
final

,Titre,Date de publication,Lien,Résumé,Mots-clés,Auteurs,Affiliations,PMID,Maladie,Langue
0,Sleep characteristics and cerebrospinal fluid ...,2020 Aug;16(8):1146-1152.,https://pubmed.ncbi.nlm.nih.gov//32657026/,Introduction:\n \n \n This st...,"Alzheimer's disease; amyloid, cerebrospinal fl...","Wei Xu, 1, Lan Tan, 1, Bing-Jie Su, 1, Huan Yu...","1 Department of Neurology, Qingdao Municipal H...",32657026,Alzheimer,en
1,Sleep Disturbance and Alzheimer's Disease: The...,2022 Jul;47(7):1799-1815.,https://pubmed.ncbi.nlm.nih.gov//35303225/,Poor quality and quantity of sleep are very co...,Alzheimer’s disease; Amyloid beta; Astrocytes;...,"Aditya Sunkaria, #, 1, Supriya Bhardwaj, #, 2","1 Department of Biotechnology, Guru Nanak Dev ...",35303225,Alzheimer,en
2,Association of Sleep With Risk of Alzheimer's ...,2022 Apr;41(4):1057-1065.,https://pubmed.ncbi.nlm.nih.gov//34109847/,Objectives:\n \n \n Alzheimer...,Alzheimer’s disease; American Association of R...,"Aaron C Schneider, 1, Chooza Moon, 1, Kara Whi...","1 University of Iowa, Iowa City, USA.; 2 The U...",34109847,Alzheimer,en
3,Sleep oscillation-specific associations with A...,2019 Feb 21;14(1):10.,https://pubmed.ncbi.nlm.nih.gov//30791922/,Background:\n \n \n Based on ...,Mots-clés non disponibles,"Korey Kam, 1, Ankit Parekh, 1, Ram A Sharma, 2...","1 Mount Sinai Integrative Sleep Center, Divisi...",30791922,Alzheimer,en
4,Diagnosis and treatment of Alzheimer's disease.,2003:64 Suppl 9:3-6.,https://pubmed.ncbi.nlm.nih.gov//12934967/,The defining characteristic of Alzheimer's dis...,Mots-clés non disponibles,"George T Grossberg, 1","1 Department of Psychiatry, St. Louis Universi...",12934967,Alzheimer,en
...,...,...,...,...,...,...,...,...,...,...
40322,"Associations of ""Weekend Warrior"" Physical Act...",2024 Oct 15;150(16):1236-1247.,https://pubmed.ncbi.nlm.nih.gov//39324186/,Background:\n \n \n Achieveme...,accelerometry; cardiovascular disease; physica...,"Shinwan Kany, 1, 2, 3, 4, Mostafa A Al-Alusi, ...","1 Department of Cardiology, University Heart a...",39324186,Obesite,en
40323,Factors that influence participation in physic...,2024 Jun 4;6(6):CD013557.,https://pubmed.ncbi.nlm.nih.gov//38837220/,Background:\n \n \n Mental he...,Mots-clés non disponibles,"Claire J McCartan, 1, Jade Yap, 2, Paul Best, ...","1 IMPACT Research Centre, Northern Health & So...",38837220,Obesite,en
40324,Recommendations from the 2023 international ev...,2023 Aug 2;189(2):G43-G64.,https://pubmed.ncbi.nlm.nih.gov//37580861/,Study question:\n \n \n What ...,GRADE; assessment; evidence based; guideline; ...,"Helena J Teede, 1, 2, Chau Thien Tay, 1, 2, Jo...",1 Monash Centre for Health Research and Implem...,37580861,Obesite,en
40325,Recommendations From the 2023 International Ev...,2023 Sep 18;108(10):2447-2469.,https://pubmed.ncbi.nlm.nih.gov//37580314/,Study question:\n \n \n What ...,GRADE; assessment; evidence-based; guideline; ...,"Helena J Teede, 1, 2, Chau Thien Tay, 1, 2, Jo...",1 Monash Centre for Health Research and Implem...,37580314,Obesite,en


#### Transformation des dates : passage au format YYYY-MM  (plus tard le jour sera ajouté lors de l'insertion dans la bd)

In [ ]:
# Dictionnaire de conversion des mois
mois_mapping = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", 
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08", 
    "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

def convertir_date(date_str):
    # Extraction de l'année
    annee_match = re.search(r"\b(\d{4})\b", date_str)
    if annee_match:
        annee = int(annee_match.group(1))  # convertion en entier plus facile à manipuler
        if annee < 2013:
            return None  # Ignorer les années < 2013 s'il y en a
    else:
        return None

    # Extraction du mois
    mois_match = re.search(r"\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b", date_str)
    mois = mois_mapping[mois_match.group(1)] if mois_match else None

    # la date finale
    if mois:
        return f"{annee}-{mois}"  # Format YYYY-MM
    return None  # Si le mois est manquant

final["Date de publication"] = final["Date de publication"].apply(convertir_date)

# Vérifions combien de dates n'ont pas été extraites
nb_dates_non_extraites = final["Date de publication"].isna().sum()
print(f"Nombre de dates non extraites : {nb_dates_non_extraites}")


Nombre de dates non extraites : 5360


In [10]:
# Supprimer les lignes où "Date de publication" est None (NaN)
final = final.dropna(subset=["Date de publication"])

final


,Titre,Date de publication,Lien,Résumé,Mots-clés,Auteurs,Affiliations,PMID,Maladie,Langue
0,Sleep characteristics and cerebrospinal fluid ...,2020-08,https://pubmed.ncbi.nlm.nih.gov//32657026/,Introduction:\n \n \n This st...,"Alzheimer's disease; amyloid, cerebrospinal fl...","Wei Xu, 1, Lan Tan, 1, Bing-Jie Su, 1, Huan Yu...","1 Department of Neurology, Qingdao Municipal H...",32657026,Alzheimer,en
1,Sleep Disturbance and Alzheimer's Disease: The...,2022-07,https://pubmed.ncbi.nlm.nih.gov//35303225/,Poor quality and quantity of sleep are very co...,Alzheimer’s disease; Amyloid beta; Astrocytes;...,"Aditya Sunkaria, #, 1, Supriya Bhardwaj, #, 2","1 Department of Biotechnology, Guru Nanak Dev ...",35303225,Alzheimer,en
2,Association of Sleep With Risk of Alzheimer's ...,2022-04,https://pubmed.ncbi.nlm.nih.gov//34109847/,Objectives:\n \n \n Alzheimer...,Alzheimer’s disease; American Association of R...,"Aaron C Schneider, 1, Chooza Moon, 1, Kara Whi...","1 University of Iowa, Iowa City, USA.; 2 The U...",34109847,Alzheimer,en
3,Sleep oscillation-specific associations with A...,2019-02,https://pubmed.ncbi.nlm.nih.gov//30791922/,Background:\n \n \n Based on ...,Mots-clés non disponibles,"Korey Kam, 1, Ankit Parekh, 1, Ram A Sharma, 2...","1 Mount Sinai Integrative Sleep Center, Divisi...",30791922,Alzheimer,en
8,Tailored music listening intervention to reduc...,2020-12,https://pubmed.ncbi.nlm.nih.gov//33136301/,Sleep disruption in older adults living with A...,Alzheimer's disease; dementia; music; sleep.,"Darina V Petrovsky, 1, Nalaka S Gooneratne, 2,...","1 Behavioral Health Sciences Department, Unive...",33136301,Alzheimer,en
...,...,...,...,...,...,...,...,...,...,...
40322,"Associations of ""Weekend Warrior"" Physical Act...",2024-10,https://pubmed.ncbi.nlm.nih.gov//39324186/,Background:\n \n \n Achieveme...,accelerometry; cardiovascular disease; physica...,"Shinwan Kany, 1, 2, 3, 4, Mostafa A Al-Alusi, ...","1 Department of Cardiology, University Heart a...",39324186,Obesite,en
40323,Factors that influence participation in physic...,2024-06,https://pubmed.ncbi.nlm.nih.gov//38837220/,Background:\n \n \n Mental he...,Mots-clés non disponibles,"Claire J McCartan, 1, Jade Yap, 2, Paul Best, ...","1 IMPACT Research Centre, Northern Health & So...",38837220,Obesite,en
40324,Recommendations from the 2023 international ev...,2023-08,https://pubmed.ncbi.nlm.nih.gov//37580861/,Study question:\n \n \n What ...,GRADE; assessment; evidence based; guideline; ...,"Helena J Teede, 1, 2, Chau Thien Tay, 1, 2, Jo...",1 Monash Centre for Health Research and Implem...,37580861,Obesite,en
40325,Recommendations From the 2023 International Ev...,2023-09,https://pubmed.ncbi.nlm.nih.gov//37580314/,Study question:\n \n \n What ...,GRADE; assessment; evidence-based; guideline; ...,"Helena J Teede, 1, 2, Chau Thien Tay, 1, 2, Jo...",1 Monash Centre for Health Research and Implem...,37580314,Obesite,en


#### Remplacer les mots-clés non disponible par les mots les plus important dans le résumé en utilisant TF-IDF

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Copie du DataFrame original
final_net = final.copy()

# Fonction pour extraire les 5 mots-clés les plus importants du résumé
def extraire_mots_cles(resume, top_n=5):
    if not isinstance(resume, str) or resume.strip() == "" or resume == "Résumé non disponible":
        return "AMC"  # Si le résumé est inexploitable

    vectorizer = TfidfVectorizer(stop_words="english") # nos articles sont en anglais
    tfidf_matrix = vectorizer.fit_transform([resume])
    
    if tfidf_matrix.shape[1] == 0:  # Aucun mot-clé trouvé
        return "AMC"
    
    # Trie des mots par score TF-IDF décroissant
    scores = tfidf_matrix.toarray()[0]
    indices = scores.argsort()[::-1][:top_n]  # Indices des top mots-clés
    
    mots_cles = [vectorizer.get_feature_names_out()[i] for i in indices if i < len(vectorizer.get_feature_names_out())]
    
    return ", ".join(mots_cles) if mots_cles else "AMC"

# Appliquer la fonction uniquement sur les articles où les mots-clés sont manquants ET où le résumé est disponible
mask = (final_net["Mots-clés"] == "Mots-clés non disponibles") & (final_net["Résumé"] != "Résumé non disponible")
final_net.loc[mask, "Mots-clés"] = final_net.loc[mask, "Résumé"].apply(extraire_mots_cles)


#### lemmatisations - stopword & Assignation des auteurs à leurs affiliations

In [ ]:
# Initialisation du lemmatizer et stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) # suppression des the, a, etc

# Fonction pour nettoyer et normaliser le texte des mots-clés
def nettoyer_mots_cles(texte):
    if not isinstance(texte, str):
        return ""
    texte = texte.lower()  # tout en minuscule
    texte = re.sub(r"\d+", "", texte)  # suppression des nombres, pas necessaire dans notre cas
    
    # Ne pas supprimer les ';' pour conserver la séparation des mots-clés entre elles
    texte = texte.translate(str.maketrans("", "", string.punctuation.replace(";", "")))
    
    # Tokenisation par ";"
    tokens = [mot.strip() for mot in texte.split(";") if mot.strip()]
    
    # Lemmatisation 
    tokens = [" ".join([lemmatizer.lemmatize(mot) for mot in mot_cle.split()]) for mot_cle in tokens]
    
    return "; ".join(tokens)  # Garder le format avec ";"

# Appliquer uniquement aux mots-clés
final_net["Mots-clés"] = final_net["Mots-clés"].astype(str).apply(nettoyer_mots_cles)



# Fonction pour extraire les auteurs et leurs affiliations
def extraire_auteurs_affiliations(auteurs_str, affiliations_str):
    if not isinstance(auteurs_str, str) or not isinstance(affiliations_str, str):
        return []
    
    auteurs_affiliations = []
    affiliations_dict = {}

    # Construction du dictionnaire des affiliations
    affiliations_list = affiliations_str.split(";") if affiliations_str else []
    for aff in affiliations_list:
        match = re.match(r"(\d+) (.+)", aff.strip())
        if match:
            affiliations_dict[match.group(1)] = match.group(2)
    
    auteurs = auteurs_str.split(", ")
    i = 0
    while i < len(auteurs):
        auteur = auteurs[i]
        i += 1
        affiliations = []
        
        while i < len(auteurs) and (auteurs[i] == "#" or auteurs[i].isdigit()):
            if auteurs[i].isdigit():
                affiliations.append(affiliations_dict.get(auteurs[i], "Affiliation inconnue"))
            i += 1
        
        auteurs_affiliations.append((auteur, affiliations))
    
    return auteurs_affiliations

# Appliquer la fonction sur auteurs et affiliations
final_net["Auteurs_Affiliations"] = final_net.apply(
    lambda row: extraire_auteurs_affiliations(row["Auteurs"], row["Affiliations"]), axis=1
)


In [ ]:
final_net.drop('Langue', axis=1,inplace=True) # suppression colonne langue qui n'est plus utile

In [ ]:
final_net.to_csv("cleaned.csv", index=False)

In [ ]:
final_net = pd.read_csv("cleaned.csv")
final_net

,Titre,Date de publication,Lien,Résumé,Mots-clés,Auteurs,Affiliations,PMID,Maladie,Auteurs_Affiliations
0,Sleep characteristics and cerebrospinal fluid ...,2020-08,https://pubmed.ncbi.nlm.nih.gov//32657026/,Introduction:\n \n \n This st...,alzheimers disease; amyloid cerebrospinal flui...,"Wei Xu, 1, Lan Tan, 1, Bing-Jie Su, 1, Huan Yu...","1 Department of Neurology, Qingdao Municipal H...",32657026,Alzheimer,"[('Wei Xu', ['Department of Neurology, Qingdao..."
1,Sleep Disturbance and Alzheimer's Disease: The...,2022-07,https://pubmed.ncbi.nlm.nih.gov//35303225/,Poor quality and quantity of sleep are very co...,alzheimer’s disease; amyloid beta; astrocyte; ...,"Aditya Sunkaria, #, 1, Supriya Bhardwaj, #, 2","1 Department of Biotechnology, Guru Nanak Dev ...",35303225,Alzheimer,"[('Aditya Sunkaria', ['Department of Biotechno..."
2,Association of Sleep With Risk of Alzheimer's ...,2022-04,https://pubmed.ncbi.nlm.nih.gov//34109847/,Objectives:\n \n \n Alzheimer...,alzheimer’s disease; american association of r...,"Aaron C Schneider, 1, Chooza Moon, 1, Kara Whi...","1 University of Iowa, Iowa City, USA.; 2 The U...",34109847,Alzheimer,"[('Aaron C Schneider', ['University of Iowa, I..."
3,Sleep oscillation-specific associations with A...,2019-02,https://pubmed.ncbi.nlm.nih.gov//30791922/,Background:\n \n \n Based on ...,tau sleep spindle csf density,"Korey Kam, 1, Ankit Parekh, 1, Ram A Sharma, 2...","1 Mount Sinai Integrative Sleep Center, Divisi...",30791922,Alzheimer,"[('Korey Kam', ['Mount Sinai Integrative Sleep..."
4,Tailored music listening intervention to reduc...,2020-12,https://pubmed.ncbi.nlm.nih.gov//33136301/,Sleep disruption in older adults living with A...,alzheimers disease; dementia; music; sleep,"Darina V Petrovsky, 1, Nalaka S Gooneratne, 2,...","1 Behavioral Health Sciences Department, Unive...",33136301,Alzheimer,"[('Darina V Petrovsky', ['Behavioral Health Sc..."
...,...,...,...,...,...,...,...,...,...,...
34962,"Associations of ""Weekend Warrior"" Physical Act...",2024-10,https://pubmed.ncbi.nlm.nih.gov//39324186/,Background:\n \n \n Achieveme...,accelerometry; cardiovascular disease; physica...,"Shinwan Kany, 1, 2, 3, 4, Mostafa A Al-Alusi, ...","1 Department of Cardiology, University Heart a...",39324186,Obesite,"[('Shinwan Kany', ['Department of Cardiology, ..."
34963,Factors that influence participation in physic...,2024-06,https://pubmed.ncbi.nlm.nih.gov//38837220/,Background:\n \n \n Mental he...,physical activity health bipolar study,"Claire J McCartan, 1, Jade Yap, 2, Paul Best, ...","1 IMPACT Research Centre, Northern Health & So...",38837220,Obesite,"[('Claire J McCartan', ['IMPACT Research Centr..."
34964,Recommendations from the 2023 international ev...,2023-08,https://pubmed.ncbi.nlm.nih.gov//37580861/,Study question:\n \n \n What ...,grade; assessment; evidence based; guideline; ...,"Helena J Teede, 1, 2, Chau Thien Tay, 1, 2, Jo...",1 Monash Centre for Health Research and Implem...,37580861,Obesite,"[('Helena J Teede', ['Monash Centre for Health..."
34965,Recommendations From the 2023 International Ev...,2023-09,https://pubmed.ncbi.nlm.nih.gov//37580314/,Study question:\n \n \n What ...,grade; assessment; evidencebased; guideline; m...,"Helena J Teede, 1, 2, Chau Thien Tay, 1, 2, Jo...",1 Monash Centre for Health Research and Implem...,37580314,Obesite,"[('Helena J Teede', ['Monash Centre for Health..."


#### Extraction pays

In [ ]:
# Dictionnaire des pays et de leurs abréviations
pays_dict = {
    "usa": "United States",
    "uk": "United Kingdom",
    "us": "United States",
    "uae": "United Arab Emirates",
    "russia": "Russian Federation"
}

# Ajouter les noms complets et leurs codes ISO
for country in pycountry.countries:
    pays_dict[country.name.lower()] = country.name  # Ex: "france" → "France"
    if hasattr(country, "alpha_2"):
        pays_dict[country.alpha_2.lower()] = country.name  # Ex: "fr" → "France"
    if hasattr(country, "alpha_3"):
        pays_dict[country.alpha_3.lower()] = country.name  # Ex: "fra" → "France"

# Fonction pour extraire le pays principal d'une affiliation
def determiner_pays(affiliations):
    pays_counts = Counter()

    # Gérer les affiliations multiples
    if isinstance(affiliations, str):
        affiliations = affiliations.split(";")

    for aff in affiliations:
        aff = aff.lower()

        # Vérification directe sur les noms et abréviations de pays
        for key, pays in pays_dict.items():
            if re.search(rf"\b{re.escape(key)}\b", aff):
                pays_counts[pays] += 1

    return pays_counts.most_common(1)[0][0] if pays_counts else "Inconnu"

# Appliquer la fonction sur pays
final_net["Pays"] = final_net["Affiliations"].apply(determiner_pays)


In [ ]:
# Afficher les premiers résultats
#clean_final[clean_final["Pays"]=="Andorra"]
#clean_final[clean_final["Pays"]=="Andorra"]["Affiliations"].iloc[35]

"1 Department of Occupational and Environmental Medicine, Wonju Severance Christian's Hospital, Yonsei University, Wonju, Republic of Korea.; 2 Department of Rehabilitation Medicine, Wonju Severance Christian's Hospital, Yonsei University, Wonju, Republic of Korea.; 3 IOC Research Centre Korea, Republic of Korea."

**remarque** : une erreur lors de la récupération des pays : plusieurs pays sont récupéré comme Andorrra

In [ ]:
final_net.to_csv("cleanedwpays.csv", index= False, encoding="utf-8")